In [55]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
from numba import config
config.DISABLE_JIT = 1 # we disable JIT compilation here, since it has quite some
# overhead for DTA that cannot be justified for toy networks
import numpy as np
import networkx as nx
import osmnx as ox
import random
from gurobipy import GRB
import gurobipy as gp


import sys
sys.path.append("..")

from dyntapy import get_toy_network
from dyntapy.visualization import show_network, show_dynamic_network, show_demand, show_link_od_flows
from dyntapy.supply_data import relabel_graph
from dyntapy.demand import DynamicDemand
from dyntapy.demand_data import od_graph_from_matrix, add_centroids
from dyntapy.demand import SimulationTime
#from dyntapy.assignments import DynamicAssignment
from osmnx.distance import euclidean_dist_vec
from dyntapy.assignments import StaticAssignment
from dyntapy.results import get_selected_link_analysis, get_od_flows

g = get_toy_network('test_g')

g = relabel_graph(g)
show_network(g, toy_network=True, notebook=True)

In [35]:
num_points = 12
#centroid_x = np.random.uniform(0, 8, num_points)
#centroid_y = np.random.uniform(0, 8, num_points)
#centroid_x = np.array([1, 3, 5, 7, 1, 3, 5, 7,-1, -1, -1, -1,9, 9, 9, 9])
#centroid_y = np.array([-1, -1, -1, -1, 9, 9, 9, 9,1, 3, 5, 7,1, 3, 5, 7])
centroid_x = np.array([1, 5, 7, 3, 5, 7, -1, -1,9, 9, 9, 9])
centroid_y = np.array([-1, -1, -1, 9, 9, 9, 5, 7,1, 3, 5, 7])
g = add_centroids(g, centroid_x, centroid_y, method="link", euclidean=True)
# also adds connectors automatically
g = relabel_graph(g)  # adding link and node ids, connectors and centroids
# are the first elements
show_network(g, euclidean=True, toy_network=True,notebook=True)

In [57]:
od_matrix = np.zeros(num_points*num_points).reshape((num_points, num_points))
for i in range(num_points):
    for j in range(num_points):
        if i!=j:
            od_matrix[i, j] = random.randint(0, 200)
od_graph = od_graph_from_matrix(od_matrix, centroid_x, centroid_y)
show_demand(od_graph, euclidean=True, toy_network=True, notebook=True)
print(od_matrix)

[[  0. 129. 138. 131. 163.  37. 186. 123. 190. 152.  76. 199.]
 [102.   0. 200. 153.  49.  82.   1.  19. 127. 126. 168. 172.]
 [ 22.  91.   0.  25.  53. 102.  77.  53. 186.  88. 161. 168.]
 [164. 136. 165.   0.  87. 107. 138. 116.   2.  67. 121. 163.]
 [143. 161. 143. 165.   0. 152.  60.  64.  11. 180.  25. 140.]
 [ 38. 183. 117. 187.  74.   0.  72. 101.  63.  60. 132.  19.]
 [ 83. 174.  15. 102.  81. 107.   0. 150. 103. 151.  49. 150.]
 [114. 146.  72.  44. 186.  72. 193.   0. 101.  94.   0. 173.]
 [ 77. 191.  66. 176.  55.  67. 158.  16.   0.  41. 183. 190.]
 [  7. 117.  73. 123. 186. 125. 143.  35.  44.   0.   5. 122.]
 [ 92. 146. 180.  10. 159. 197. 197.  23. 119.  98.   0.  12.]
 [145.  85. 130. 171. 124.  21. 181. 199.  75.  82.  48.   0.]]


In [39]:
assignment = StaticAssignment(g,od_graph)
#methods = ['msa', 'sun', 'dial_b']
methods = ['dial_b']
for method in methods:
    result = assignment.run(method)
    show_network(g, flows = result.flows, euclidean=True, notebook=True, show_nodes=True)
    print(f'{method=} ran successfully')    
# all static assignments return a result object that follows the same structure, see
# below
print(result.__dict__.keys())

init passed successfully
solution found, Dial B in iteration 4


method='dial_b' ran successfully
dict_keys(['link_costs', 'flows', 'origins', 'destinations', 'origin_flows', 'destination_flows', 'skim', 'gap_definition', 'gap', 'od_flows'])


In [53]:
print(result.flows[1])
print(result.flows)
print(len(result.flows))

1119.0
[ 877.         1119.         1168.         1591.         1045.
  870.          781.         1385.         1017.         1040.
 1089.          851.          544.          578.579592   1183.
  724.36079691  722.63974905  668.97735778  902.10172119  551.07476593
  610.87869923 1150.          988.75377521  582.56435099 1094.
  477.47261614  704.21879508  875.93167825  705.3541313   673.84436451
  669.87618256  823.81909204  912.10473221  531.48958505  721.66166209
  850.15150304 1009.7128491   729.24336027  988.09772667  625.
  770.52738386  773.68936618 1021.5790925  1099.          782.86416429
  715.51588644  814.33906428  932.39687736  898.18523549  780.68469936
  720.97156955  853.7749798   898.23543477  647.          980.24905034
  933.         1017.          887.          927.         1291.
  915.4209075  1027.          752.58428097 1234.          939.60831112
  973.75094966  789.71365994 1094.          827.29794091  644.70205909
 1045.         1158.        ]
72


In [59]:
num_roads = len(result.flows)
c_a = 2000
l = 2
delta_a = 2
max_expansions = 10
model = gp.Model("traffic_optimization_with_expansion")
y = model.addVars(num_roads, vtype=GRB.BINARY, name="y")
t_exp_a = [model.addVar(name=f"t_exp_{a}") for a in range(num_roads)]
def calculate_ta(x, c, l, delta, y):
    return (y * (1 + 0.15 * (x / (c * (l + delta))) ** 4) + 
            (1 - y) * (1 + 0.15 * (x / (c * l)) ** 4))
model.setObjective(gp.quicksum(result.flows[a] * t_exp_a[a] for a in range(num_roads)), GRB.MINIMIZE)
for a in range(num_roads):
    model.addConstr(
        t_exp_a[a] == calculate_ta(result.flows[a], c_a, l, delta_a, y[a]), 
        f"expansion_impact_{a}"
    )
model.addConstr(gp.quicksum(y[a] for a in range(num_roads)) <= max_expansions, "max_expansions")

model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.objVal}")
    for a in range(num_roads):
        print(f"Road {a}: x = {result.flows[a]:.2f}, t_exp = {t_exp_a[a].x:.2f}, y = {y[a].x}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11400F @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 73 rows, 144 columns and 216 nonzeros
Model fingerprint: 0x5af74623
Variable types: 72 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+00]
  Objective range  [5e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 63615.809274
Presolve removed 73 rows and 144 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 63597.3 63615.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.359730827750e+04, best bound 6.359730827750e+04, gap 0.0000%
Opti

In [45]:
quadratic_model = Model('quadratic')
x = quadratic_model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = "x")
y = quadratic_model.addVar(vtype = GRB.BINARY, name = "y")
obj_fn = 0
for flow in result.flows:
    t_a = t_fa * (1 + 0.15 * (flow/2000)**4) + 
    obj_fn = obj_fn + flow * 

Set parameter Username


In [59]:
origins = result.origins
destinations = result.destinations
flows = result.flows
od_flows = result.od_flows
print(origins)
print(destinations)
print(flows)
print(od_flows)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
[ 467.          416.          534.          548.          495.
  500.          501.          473.          455.          494.
  469.          437.          468.          437.          500.
  508.          523.          440.          512.          481.
  513.          425.          373.          411.          528.
  482.          551.          489.          529.          518.
  530.          439.          524.          471.          478.
  491.          477.          526.          488.          563.
  488.          548.          495.          533.          545.
  503.          501.          450.          476.          575.
  846.31197602  935.12698138  475.          462.          922.

In [ ]:
od_flows = get_od_flows(assignment, result)
print(od_flows)
show_link_od_flows(g, od_flows, euclidean=True, notebook=True, show_nodes=False)